<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/hak_1_KoBERT_dialect_dataset_label_generation_cc_70000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_70000_std.pt'

#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.9 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 11.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595742 sha256=7b4c00af2c220a020885e03388dc2dfb0d4171bead900e5c35542feb449e7cce
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 11.2 MB/s 
     |████████████████████████████████| 769 kB 9.2 MB/s 
     |████████████████████████████████| 895 kB 12.3 MB/s 
     |████████████████████████████████| 3.0 MB 45.5 MB/s 


#KoBERT 다운로드(사용)#

In [4]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-u5lp_2q0
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-u5lp_2q0
     |████████████████████████████████| 4.9 MB 13.0 MB/s 
     |████████████████████████████████| 3.4 MB 37.3 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 61 kB 489 kB/s 
     |████████████████████████████████| 3.3 MB 26.0 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=9bec45ab8215d5a3a07d7bc5bcb06a03d33b5caf50a89a2ed7ce6958059b8b74
  Stored in directory: /tmp/pip-ephem-wheel-cache-15hwpa43/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found exist

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#GPU 사용
device = torch.device("cuda:0")

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#KoBERT 입력 데이터로 변환(사용)#

In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#원본 데이터셋으로부터 데이터 추출하기#

In [11]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [12]:
!echo $VALID_DIR

/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/valid


In [13]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep json | grep -v data | grep -v answer| grep -v config').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/train
/content
popd -> /content


In [14]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep json | grep -v data | grep -v answer | grep -v config').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/valid
/content
popd -> /content


In [15]:
len(train_json_list)

5651

In [16]:
len(valid_json_list)

780

#추출한 데이터에 감정 레이블 추가(사용)#

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [18]:
#모델 불러오기
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


In [20]:
import csv

def GenLabelFromData70000(path, jsonfilename):
  try:
    fin = open('%s/%s.data' % (path, jsonfilename), 'r')
  except:
    print('No file(%s/%s.data)' % (path, jsonfilename))
    return

  with open('%s/%s.data' % (path, jsonfilename), 'r') as f:
    with open('%s/%s.answer70000' % (path, jsonfilename), 'w') as o:
      csv_data = csv.reader(f, delimiter='\t')
      for i, line in enumerate(csv_data):
        std = line[0]
        dia = line[1]
        idx, val = predict(line[0])
        #print('%s(%d) %s\n' % (val, idx, std))
        #print('[%d]%s/%s/%d/%s\n' % (i, std, dia, idx, val))
        o.write('%s\t%s\t%d\n' % (std, dia, idx))
  print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [21]:
train_json_list[0:171]

['DCDG20000001.json',
 'DCDG20000002.json',
 'DCDG20000003.json',
 'DCDG20000004.json',
 'DCDG20000005.json',
 'DCDG20000006.json',
 'DCDG20000007.json',
 'DCDG20000008.json',
 'DCDG20000403.json',
 'DCDG20000404.json',
 'DCDG20000405.json',
 'DCDG20000406.json',
 'DCDG20000407.json',
 'DCDG20000408.json',
 'DCDG20000409.json',
 'DCDG20000410.json',
 'DCDG20000411.json',
 'DCDG20000412.json',
 'DCDG20000413.json',
 'DCDG20000414.json',
 'DCDG20000415.json',
 'DCDG20000416.json',
 'DCDG20000417.json',
 'DCDG20000418.json',
 'DCDG20000419.json',
 'DCDG20000420.json',
 'DCDG20000421.json',
 'DCDG20000422.json',
 'DCDG20000423.json',
 'DCDG20000424.json',
 'DCDG20000425.json',
 'DCDG20000426.json',
 'DCDG20000427.json',
 'DCDG20000428.json',
 'DCDG20000429.json',
 'DCDG20000430.json',
 'DCDG20000431.json',
 'DCDG20000432.json',
 'DCDG20000433.json',
 'DCDG20000434.json',
 'DCDG20000435.json',
 'DCDG20000436.json',
 'DCDG20000437.json',
 'DCDG20000438.json',
 'DCDG20000439.json',
 'DCDG2000

In [22]:
valid_json_list[0:58]

['DCDG20000009.json',
 'DCDG20000010.json',
 'DCDG20000011.json',
 'DCDG20000012.json',
 'DCDG20000013.json',
 'DCDG20000014.json',
 'DCDG20000015.json',
 'DCDG20000016.json',
 'DCDG20000017.json',
 'DCDG20000018.json',
 'DCDG20000019.json',
 'DCDG20000020.json',
 'DCDG20000021.json',
 'DCDG20000022.json',
 'DCDG20000023.json',
 'DCDG20000024.json',
 'DCDG20000025.json',
 'DCDG20000026.json',
 'DCDG20000027.json',
 'DCDG20000028.json',
 'DCDG20000029.json',
 'DCDG20000030.json',
 'DCDG20000031.json',
 'DCDG20000032.json',
 'DCDG20000033.json',
 'DCDG20000034.json',
 'DCDG20000035.json',
 'DCDG20000036.json',
 'DCDG20000037.json',
 'DCDG20000038.json',
 'DCDG20000039.json',
 'DCDG20000040.json',
 'DCDG20000041.json',
 'DCDG20000042.json',
 'DCDG20000043.json',
 'DCDG20000044.json',
 'DCDG20000045.json',
 'DCDG20000046.json',
 'DCDG20000047.json',
 'DCDG20000048.json',
 'DCDG20000049.json',
 'DCDG20000050.json',
 'DCDG20000051.json',
 'DCDG20000052.json',
 'DCDG20000053.json',
 'DCDG2000

In [26]:
# for문을 돌면서 json 파일 내에 있는 표준어 문장과 사투리 문장을 추출해서 FILENAME.data로 저장
import json

def GenDataFromJson(path, jsonfilename):
  with open('%s/%s' % (path, jsonfilename), 'r') as f:
    json_data = json.load(f);
  #print(json.dumps(json_data, indent='\t'))
  ut = json_data['utterance']

  with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
    for u in ut:
      f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))
   
%pushd $TRAIN_DIR
# 450개 파일 * 350개 문장
#for f in train_json_list[0:171]:
#  GenDataFromJson(TRAIN_DIR, f)
#%popd

#%pushd $VALID_DIR
# 150개 파일 350개 문장
#for f in valid_json_list[0:58]:
#  GenDataFromJson(VALID_DIR, f)
%popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/train
/content
popd -> /content
/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/valid
/content
popd -> /content


In [ ]:
%pushd $TRAIN_DIR
# 450개 파일 350개 문장
#for i, f in enumerate(train_json_list[0:100]):
#  print('[%d][%s]' % (i, f))
#  GenLabelFromData70000(TRAIN_DIR, f)
#popd

/content/drive/.shortcut-targets-by-id/1-0pJ0cGHz95nSQSccZedRVC_hwqx-59q/KAIST/SEP531/kor_cc_labeled/train
[0][DCDG20000001.json]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000001.json.answer

[1][DCDG20000002.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000002.json.answer

[2][DCDG20000003.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000003.json.answer

[3][DCDG20000004.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000004.json.answer

[4][DCDG20000005.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000005.json.answer

[5][DCDG20000006.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000006.json.answer

[6][DCDG20000007.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000007.json.answer

[7][DCDG20000008.json]
>> Generated file /content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train/DCDG20000008.json.answer

[8][DCDG20000403.json]
>> Gener

In [ ]:
%pushd $TRAIN_DIR
# 450개 파일 350개 문장
for i, f in enumerate(train_json_list[30:171]):
  print('[%d][%s]' % (i, f))
  GenLabelFromData70000(TRAIN_DIR, f)
%popd

In [ ]:
%pushd $TRAIN_DIR
# 450개 파일 350개 문장
for i, f in enumerate(train_json_list[101:171]):
  print('[%d][%s]' % (i, f))
  GenLabelFromData70000(TRAIN_DIR, f)
%popd

In [ ]:
%pushd $VALID_DIR
# 450개 파일 350개 문장
for i, f in enumerate(valid_json_list[0:58]):
  print('[%d][%s]' % (i, f))
  GenLabelFromData70000(VALID_DIR, f)
%popd